In [1]:
# Попробуем построить модель только для легковых ТС, которыми владеют физлица, убрав ряд фичей,
# оказавшихся неактуальными в model2
# готовим модель для CatBoost

import pandas as pd
import random as r
import numpy as np
import datetime
from sklearn import preprocessing
from tqdm import tqdm
tqdm.pandas(desc="")

C:\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
dt = pd.read_csv('../data/primary.csv')
# откуда то при пересохранении в csv образуется лишний столбец
# dt.drop(dt.columns[[0]], axis = 1, inplace = True)

# выводим отдельным признаком наличие убытков
dt['claim'] = dt['CNT_CLAIM_PAY'].apply(lambda x: 0 if x == 0 else 1)

# оставляем только легковые авто от физлиц
dt = dt[(dt['TS_TYPE'] == 'Легковые автомобили') 
   & (dt['OWN_JUR'] == 0) 
   & (dt['INS_JUR'] == 0)].drop(['TS_TYPE', 'OWN_JUR', 'INS_JUR'], axis=1).drop(dt.columns[[0]], axis = 1)

dt.head()

,AGR_ISN,AGR_DATESIGN,AGR_DATEBEG,AGR_DATEEND,AGR_SRC,AGR_CREATEDBY,INS_ISN,INS_NAME,INS_SEX,INS_YEAR,...,CNT_PVU_NOT_PAY,CNT_PVU_PAY,SUM_PVU_NOT_PAY,SUM_PVU_PAY,COURT_EXISTS,COURT_AMOUNT,INS_ANTI_VIP,OWN_ANTI_VIP,DRV_ANTI_VIP,claim
1,886049726,13.02.15,16.02.15,15.02.16,КИАС:Страхование,Карабегович Алла Викторовна,126439000,Корендясев Алексей Юрьевич,М,49.0,...,0,0,NaN,NaN,0,NaN,0,0,"0,0,0",0
2,886482027,25.09.15,30.09.15,29.09.16,КИАС:Страхование,Карабегович Алла Викторовна,157142000,Гребнева Галина Ивановна,Ж,52.0,...,0,0,NaN,NaN,0,NaN,0,0,"0,0,0,0,0",0
3,886618143,25.09.15,30.09.15,29.09.16,КИАС:Страхование,Попов Алексей Сергеевич,157142000,Гребнева Галина Ивановна,Ж,52.0,...,0,0,NaN,NaN,0,NaN,0,0,"0,0,0,0",0
4,886683292,10.02.16,16.02.16,15.02.17,КИАС:Страхование,Карабегович Алла Викторовна,126439000,Корендясев Алексей Юрьевич,М,50.0,...,0,0,NaN,NaN,0,NaN,0,0,"0,0,0,0",0
5,886922224,20.09.16,30.09.16,29.09.17,КИАС:Страхование,Брук Борис Борисович,157142000,Гребнева Галина Ивановна,Ж,53.0,...,0,0,NaN,NaN,0,NaN,0,0,"0,0,0,0,0",0


In [3]:
# убираем лишние столбцы

dt.drop(['AGR_ISN',
         'COURT_EXISTS',
         'COURT_AMOUNT',
         'INS_ANTI_VIP',
         'OWN_ANTI_VIP',
         'DRV_ANTI_VIP',
         'INS_ISN',
         'OWN_ISN'
        ], axis = 1, inplace = True)

dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81971 entries, 1 to 107080
Data columns (total 44 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   AGR_DATESIGN       81971 non-null  object 
 1   AGR_DATEBEG        81971 non-null  object 
 2   AGR_DATEEND        81971 non-null  object 
 3   AGR_SRC            81971 non-null  object 
 4   AGR_CREATEDBY      81971 non-null  object 
 5   INS_NAME           81971 non-null  object 
 6   INS_SEX            80480 non-null  object 
 7   INS_YEAR           80575 non-null  float64
 8   INS_REG            79912 non-null  float64
 9   INS_DOCSER         80579 non-null  object 
 10  OWN_NAME           81971 non-null  object 
 11  OWN_SEX            80480 non-null  object 
 12  OWN_YEAR           80575 non-null  float64
 13  OWN_REG            80017 non-null  float64
 14  OWN_DOCSER         80579 non-null  object 
 15  DRV_NAME           75755 non-null  object 
 16  DRV_SEX            75

# План работ
Построим модель, предсказывающую вероятность убытка по параметрам договора

+ из дат подписания и начала можно получить информацию о том, насколько заранее куплен полис
+ из источника продаж, создателя, категории, марки, и популярных моделей получим среднюю убыточность по каждому из них
+ из года выпуска получим возраст машины на момент страхования
+ по каждому региону (собственника, страхователя, + по их паспортам) также получим среднюю убыточность
+ данных о регионах очень мало, но они очень важны. Попробуем заполнить недостающие данные о регионе использования ТС с помощью региона выдачи паспорта. Потом получим среднюю убыточность по региону
+ выведем отдельно признак пролонгируемого договора
+ пол закодируем в onehot, где он неизвестен (премущественно это мультидрайв) будет 0,0
+ отдельными колонками выведем общее количество водителей, минимальный и максимальный возраст, минимальный и максимальный КБМ

In [4]:
# из дат подписания и начала можно получить информацию о том, насколько заранее куплен полис

def daysbetween(d1, d2):
    dt1 = datetime.datetime.strptime(d1, '%d.%m.%y')
    dt2 = datetime.datetime.strptime(d2, '%d.%m.%y')
    return (dt2 - dt1).days

dt['f1_daysbeforestart'] = dt.apply(lambda x: daysbetween(x['AGR_DATESIGN'], x['AGR_DATEBEG']), axis=1)

In [5]:
dt.claim.value_counts()

0    78204
1     3767
Name: claim, dtype: int64

In [6]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81971 entries, 1 to 107080
Data columns (total 45 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AGR_DATESIGN        81971 non-null  object 
 1   AGR_DATEBEG         81971 non-null  object 
 2   AGR_DATEEND         81971 non-null  object 
 3   AGR_SRC             81971 non-null  object 
 4   AGR_CREATEDBY       81971 non-null  object 
 5   INS_NAME            81971 non-null  object 
 6   INS_SEX             80480 non-null  object 
 7   INS_YEAR            80575 non-null  float64
 8   INS_REG             79912 non-null  float64
 9   INS_DOCSER          80579 non-null  object 
 10  OWN_NAME            81971 non-null  object 
 11  OWN_SEX             80480 non-null  object 
 12  OWN_YEAR            80575 non-null  float64
 13  OWN_REG             80017 non-null  float64
 14  OWN_DOCSER          80579 non-null  object 
 15  DRV_NAME            75755 non-null  object 
 16  DRV

In [7]:
# из года выпуска получим возраст машины на момент страхования


dt['f6_car_age_on_start'] = dt.apply(lambda x: datetime.datetime.strptime(x['AGR_DATEBEG'], '%d.%m.%y').year - x['TS_YEAR'], axis=1)

In [8]:
# КБМ по договорам с мультидрайвом отсутствует, заполним его медианой КБМ по другим договорам
dt['AGR_KBM'].fillna(dt['AGR_KBM'].median(), inplace=True)
dt.AGR_KBM.unique()

array([0.85, 0.8 , 1.  , 0.75, 0.95, 0.7 , 0.9 , 0.65, 0.6 , 0.5 , 0.55,
       1.4 , 1.55, 2.45, 2.3 , 1.04])

In [9]:
# выведем отдельно признак пролонгируемого договора
dt['f13_is_prolongation'] = dt['AGR_PREVISN'].apply(lambda x: 1 if x>0 else 0)
dt.f13_is_prolongation.unique()

array([1, 0], dtype=int64)

In [10]:
# отдельными колонками выведем общее количество водителей
dt['DRV_ISN'].fillna(0, inplace=True)
dt['f14_drivers_count'] = dt['DRV_ISN'].apply(lambda x: 0 if x == 0 else len(str(x).split(',')))
dt.f14_drivers_count.unique()

array([ 3,  5,  4,  1,  2,  0,  6,  7,  8,  9, 19, 18, 10, 11, 12, 15, 13,
       16, 17], dtype=int64)

In [11]:
dt.groupby('f14_drivers_count').count()

,AGR_DATESIGN,AGR_DATEBEG,AGR_DATEEND,AGR_SRC,AGR_CREATEDBY,INS_NAME,INS_SEX,INS_YEAR,INS_REG,INS_DOCSER,...,SUM_CLAIM_NOT_PAY,SUM_CLAIM_PAY,CNT_PVU_NOT_PAY,CNT_PVU_PAY,SUM_PVU_NOT_PAY,SUM_PVU_PAY,claim,f1_daysbeforestart,f6_car_age_on_start,f13_is_prolongation
f14_drivers_count,,,,,,,,,,,,,,,,,,,,,
0,6216,6216,6216,6216,6216,6216,6059,6091,6090,6087,...,1204,1224,6216,6216,1209,1224,6216,6216,6215,6216
1,31603,31603,31603,31603,31603,31603,31108,31151,30812,31150,...,4326,4378,31603,31603,4330,4378,31603,31603,31603,31603
2,27915,27915,27915,27915,27915,27915,27465,27476,27207,27474,...,3720,3773,27915,27915,3728,3773,27915,27915,27915,27915
3,10738,10738,10738,10738,10738,10738,10486,10492,10444,10497,...,1294,1312,10738,10738,1300,1312,10738,10738,10738,10738
4,4190,4190,4190,4190,4190,4190,4089,4090,4089,4093,...,511,526,4190,4190,515,526,4190,4190,4190,4190
5,939,939,939,939,939,939,912,914,910,917,...,129,130,939,939,129,130,939,939,939,939
6,236,236,236,236,236,236,230,230,231,230,...,26,28,236,236,27,28,236,236,236,236
7,75,75,75,75,75,75,72,72,74,72,...,9,9,75,75,9,9,75,75,75,75
8,29,29,29,29,29,29,29,29,26,29,...,4,4,29,29,4,4,29,29,29,29


In [12]:
def minmaxfunc(x):
    l = str(x).split(',')
    lf = [float(i) for i in l]
    return lf

minmaxfunc('1,3,4,5')

[1.0, 3.0, 4.0, 5.0]

In [13]:
# отдельными колонками выведем минимальный и максимальный возраст, стаж, и КБМ 
# (КБМ только мин, макс и так в договоре есть)
dt['DRV_KBM'].fillna(dt['AGR_KBM'].median(), inplace=True)
dt['f15_kbm_min'] = dt['DRV_KBM'].apply(lambda x: min(minmaxfunc(x)))

In [14]:
dt['f16_drv_age_min'] = dt['DRV_YEAR'].apply(lambda x: min(minmaxfunc(x)))
dt['f17_drv_age_max'] = dt['DRV_YEAR'].apply(lambda x: max(minmaxfunc(x)))
dt['f18_drv_skill_min'] = dt['DRV_SKILL'].apply(lambda x: min(minmaxfunc(x)))
dt['f19_drv_skill_max'] = dt['DRV_SKILL'].apply(lambda x: max(minmaxfunc(x)))

In [15]:
# дополнительно выведем информацию о том, что собственник = страхователю и он же водит

def oneperson(s):
    if s['INS_NAME'] == s['OWN_NAME'] and s['OWN_NAME'] in str(s['DRV_NAME']).split(','):
        return 1
    else:
        return 0

In [16]:
dt['f20_oneperson'] = dt.apply(oneperson, axis=1)

In [17]:
# забыл вывести длительность договора
dt['f21_agr_days'] = dt.apply(lambda x: daysbetween(x['AGR_DATEBEG'], x['AGR_DATEEND']), axis=1)

In [18]:
# после каждого шага сохраняем датасет. Крайняя фича - f20
dt.to_csv('../data/v4step1.csv')

In [19]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81971 entries, 1 to 107080
Data columns (total 55 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   AGR_DATESIGN         81971 non-null  object 
 1   AGR_DATEBEG          81971 non-null  object 
 2   AGR_DATEEND          81971 non-null  object 
 3   AGR_SRC              81971 non-null  object 
 4   AGR_CREATEDBY        81971 non-null  object 
 5   INS_NAME             81971 non-null  object 
 6   INS_SEX              80480 non-null  object 
 7   INS_YEAR             80575 non-null  float64
 8   INS_REG              79912 non-null  float64
 9   INS_DOCSER           80579 non-null  object 
 10  OWN_NAME             81971 non-null  object 
 11  OWN_SEX              80480 non-null  object 
 12  OWN_YEAR             80575 non-null  float64
 13  OWN_REG              80017 non-null  float64
 14  OWN_DOCSER           80579 non-null  object 
 15  DRV_NAME             75755 non-null

In [20]:
dt['INS_YEAR'].fillna(dt['INS_YEAR'].median(), inplace=True)
dt['OWN_YEAR'].fillna(dt['OWN_YEAR'].median(), inplace=True)
dt['f18_drv_skill_min'].fillna(dt['f18_drv_skill_min'].median(), inplace=True)
dt['f19_drv_skill_max'].fillna(dt['f19_drv_skill_max'].median(), inplace=True)
dt['f16_drv_age_min'].fillna(dt['f16_drv_age_min'].median(), inplace=True)
dt['f17_drv_age_max'].fillna(dt['f17_drv_age_max'].median(), inplace=True)

In [21]:
print(dt.columns)

Index(['AGR_DATESIGN', 'AGR_DATEBEG', 'AGR_DATEEND', 'AGR_SRC',
       'AGR_CREATEDBY', 'INS_NAME', 'INS_SEX', 'INS_YEAR', 'INS_REG',
       'INS_DOCSER', 'OWN_NAME', 'OWN_SEX', 'OWN_YEAR', 'OWN_REG',
       'OWN_DOCSER', 'DRV_NAME', 'DRV_SEX', 'DRV_YEAR', 'DRV_DOCSER',
       'DRV_KBM', 'DRV_SKILL', 'AGR_KBM', 'DRV_ISN', 'TS_CATEGORY',
       'TS_MARK_NAME', 'TS_MODEL_NAME', 'TS_YEAR', 'TS_VIN', 'TS_POWER',
       'TS_REGNO', 'TS_USE_NAME', 'TS_DOC', 'TS_DOC_SER_NO', 'AGR_PREVISN',
       'AGR_NEXTISN', 'CNT_CLAIM_NOT_PAY', 'CNT_CLAIM_PAY',
       'SUM_CLAIM_NOT_PAY', 'SUM_CLAIM_PAY', 'CNT_PVU_NOT_PAY', 'CNT_PVU_PAY',
       'SUM_PVU_NOT_PAY', 'SUM_PVU_PAY', 'claim', 'f1_daysbeforestart',
       'f6_car_age_on_start', 'f13_is_prolongation', 'f14_drivers_count',
       'f15_kbm_min', 'f16_drv_age_min', 'f17_drv_age_max',
       'f18_drv_skill_min', 'f19_drv_skill_max', 'f20_oneperson',
       'f21_agr_days'],
      dtype='object')


In [22]:
data = dt[['AGR_SRC',
       'AGR_CREATEDBY', 'INS_NAME', 'INS_SEX', 'INS_YEAR', 'INS_REG',
       'INS_DOCSER', 'OWN_NAME', 'OWN_SEX', 'OWN_YEAR', 'OWN_REG',
       'OWN_DOCSER', 'AGR_KBM', 'TS_CATEGORY',
       'TS_MARK_NAME', 'TS_MODEL_NAME', 'TS_POWER', 'TS_USE_NAME', 'TS_DOC', 'claim', 'f1_daysbeforestart',
       'f6_car_age_on_start', 'f13_is_prolongation', 'f14_drivers_count',
       'f15_kbm_min', 'f16_drv_age_min', 'f17_drv_age_max',
       'f18_drv_skill_min', 'f19_drv_skill_max', 'f20_oneperson',
       'f21_agr_days']]

data.head(20)

,AGR_SRC,AGR_CREATEDBY,INS_NAME,INS_SEX,INS_YEAR,INS_REG,INS_DOCSER,OWN_NAME,OWN_SEX,OWN_YEAR,...,f6_car_age_on_start,f13_is_prolongation,f14_drivers_count,f15_kbm_min,f16_drv_age_min,f17_drv_age_max,f18_drv_skill_min,f19_drv_skill_max,f20_oneperson,f21_agr_days
1,КИАС:Страхование,Карабегович Алла Викторовна,Корендясев Алексей Юрьевич,М,49.0,45.0,4611,Гребнева Галина Ивановна,М,49.0,...,8.0,1,3,0.85,29.0,51.0,10.0,28.0,0,364
2,КИАС:Страхование,Карабегович Алла Викторовна,Гребнева Галина Ивановна,Ж,52.0,45.0,4510,Гребнева Галина Ивановна,Ж,52.0,...,8.0,1,5,0.70,30.0,52.0,11.0,29.0,1,365
3,КИАС:Страхование,Попов Алексей Сергеевич,Гребнева Галина Ивановна,Ж,52.0,45.0,4510,Гребнева Галина Ивановна,Ж,52.0,...,8.0,0,4,0.80,30.0,52.0,11.0,29.0,1,365
4,КИАС:Страхование,Карабегович Алла Викторовна,Корендясев Алексей Юрьевич,М,50.0,45.0,4611,Гребнева Галина Ивановна,М,50.0,...,9.0,1,4,0.80,23.0,52.0,0.0,29.0,0,365
5,КИАС:Страхование,Брук Борис Борисович,Гребнева Галина Ивановна,Ж,53.0,45.0,4510,Гребнева Галина Ивановна,Ж,53.0,...,9.0,1,5,0.70,31.0,53.0,12.0,30.0,1,364
6,КИАС:Страхование,Краснова Юлия Анатольевна,Корендясев Алексей Юрьевич,М,51.0,45.0,4611,Гребнева Галина Ивановна,М,51.0,...,10.0,1,4,0.75,24.0,53.0,1.0,30.0,0,364
7,КИАС:Страхование,Брук Борис Борисович,Корендясев Алексей Юрьевич,М,51.0,45.0,4611,Гребнева Галина Ивановна,М,51.0,...,10.0,0,4,0.75,24.0,53.0,1.0,30.0,0,364
8,КИАС:Страхование,Брук Борис Борисович,Гребнева Галина Ивановна,Ж,54.0,45.0,4510,Гребнева Галина Ивановна,Ж,54.0,...,10.0,1,5,0.65,32.0,54.0,13.0,31.0,1,364
9,КИАС:Страхование,Бодриков Дмитрий Георгиевич,Корендясев Алексей Юрьевич,М,52.0,45.0,4611,Гребнева Галина Ивановна,М,52.0,...,11.0,1,4,0.70,25.0,54.0,2.0,31.0,0,364
10,КИАС:Страхование,Брук Борис Борисович,Гребнева Галина Ивановна,Ж,55.0,45.0,4510,Гребнева Галина Ивановна,Ж,55.0,...,11.0,1,5,0.60,33.0,55.0,14.0,32.0,1,364


In [23]:
# сохраним итоговый вариант
data.to_csv('../data/model4.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81971 entries, 1 to 107080
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   AGR_SRC              81971 non-null  object 
 1   AGR_CREATEDBY        81971 non-null  object 
 2   INS_NAME             81971 non-null  object 
 3   INS_SEX              80480 non-null  object 
 4   INS_YEAR             81971 non-null  float64
 5   INS_REG              79912 non-null  float64
 6   INS_DOCSER           80579 non-null  object 
 7   OWN_NAME             81971 non-null  object 
 8   OWN_SEX              80480 non-null  object 
 9   OWN_YEAR             81971 non-null  float64
 10  OWN_REG              80017 non-null  float64
 11  OWN_DOCSER           80579 non-null  object 
 12  AGR_KBM              81971 non-null  float64
 13  TS_CATEGORY          81971 non-null  object 
 14  TS_MARK_NAME         81971 non-null  object 
 15  TS_MODEL_NAME        81971 non-null